In [ ]:
import sys
from utils.datasets import Datasets
from utils.parametric_qsne import ParametricQSNE
from utils.save_results import saveInfo, plotGraph, plotScatter
from utils.utils import dict2PCA, dict2kNN, numpy2dict
from utils.networks import *

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

import numpy as np

In [ ]:
device = torch.device("cuda:0")

DATASET = 'MNIST'
BATCH_SIZE = 5000
NUM_WORKERS = 2

WEIGHT_DECAY = 0.0001
LEARNING_RATE = 0.05
STEPS = 10
GAMMA = 0.9

SEED = 1

EPOCH = 100

DIM = 2
PERPLEXITY = 30.0
q = 2.0

K = 5

In [ ]:
torch.cuda.manual_seed_all(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
train_trans = transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize((0.485, 0.456, 0.406),
            #                     (0.229, 0.224, 0.225)),
        ])
test_trans = transforms.Compose([
            transforms.ToTensor(),
            #transforms.Normalize((0.485, 0.456, 0.406),
            #                     (0.229, 0.224, 0.225)),
        ])
instance_datasets = Datasets(DATASET,
                             BATCH_SIZE,
                             NUM_WORKERS,
                             transform=train_trans,
                             test_transform=test_trans,
                            )
data_sets = instance_datasets.create()

trainloader = data_sets[0]
testloader = data_sets[1]
classes = data_sets[2]
based_labels = data_sets[3]
input_channels = data_sets[4]
trainset = data_sets[5]
testset = data_sets[6]

In [ ]:
model = Net(input_channels=input_channels,
            num_classes=DIM).to(device)

#optimizer = optim.SGD(model.parameters(),
#                      lr=LEARNING_RATE,
#                      momentum=0.9,
#                      weight_decay=WEIGHT_DECAY)
optimizer = optim.Adam(model.parameters(),
                       lr=LEARNING_RATE)

#scheduler = optim.lr_scheduler.StepLR(optimizer,
#                                      step_size=STEPS,
#                                      gamma=GAMMA)
scheduler = None

criterion = nn.KLDivLoss(reduction="sum")

pqsne = ParametricQSNE(model,
                        optimizer,
                        criterion,
                        q,
                        PERPLEXITY)

In [ ]:
losses = pqsne.fit(EPOCH,
                    trainloader,
                    scheduler=scheduler,
                    device=device,
                    )

In [ ]:
plotGraph(losses, 5.0, filename="loss")

In [ ]:
info = {}
info["DATASET"] = DATASET
info["Learnign_rate"] = LEARNING_RATE
info["Weight_decay"] = WEIGHT_DECAY
info["EPOCH"] = EPOCH
info["STEPS"] = STEPS
info["GAMMA"] = GAMMA
info["q"] = q
info["PERPLEXITY"] = PERPLEXITY
info["DIM"] = DIM
info["K"] = K
info["Model"] = model
info["Criterion"] = criterion
info["Optimizer"] = optimizer
info["train_loss"] = losses["train"][-1]

In [ ]:
saveInfo(info, DATASET+"_info")

In [ ]:
torch.save(model.state_dict(), 'param.pth')
torch.save(model, 'model.pth')

In [ ]:
train_embedding = pqsne.getOutputs(trainloader, based_labels)
test_embedding = pqsne.getOutputs(testloader, based_labels)

In [ ]:
#train_embedding, test_embedding, pca = dict2PCA(train_embedding,
#                                                test_embedding,
#                                                dim=2)

In [ ]:
plotScatter(train_embedding, s=1, filename="train_embeddings")
plotScatter(test_embedding, s=1, filename="test_embeddings")

In [ ]:
train_score, kNN = dict2kNN(train_embedding, K)
test_score, _ = dict2kNN(test_embedding, K, kNN)
print(train_score, test_score)